In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *

In [3]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [4]:
PATH = "data/hackerearth-myntra/"
# sz=224

In [5]:
import pandas as pd

In [6]:
df_raw = pd.read_csv(f'{PATH}/csv/myntra_train_dataset.csv', low_memory=False)

In [7]:
df_raw.head()

,Brand,Category,Gender,Color,Link_to_the_image,Sub_category
0,Roadster,Tshirts,Men,Black,http://myntra.myntassets.com/assets/images/182...,Graphic
1,Roadster,Tshirts,Men,Black,http://myntra.myntassets.com/assets/images/182...,Graphic
2,Roadster,Tshirts,Men,Black,http://myntra.myntassets.com/assets/images/182...,Biker
3,Roadster,Tshirts,Men,Black,http://myntra.myntassets.com/assets/images/182...,Graphic
4,Roadster,Tshirts,Men,Black,http://myntra.myntassets.com/assets/images/182...,Graphic


In [10]:
import numpy as np

np.unique(df_raw['Sub_category'])

array(['Abstract', 'Biker', 'Camouflage', 'Checked', 'Colourblocked', 'Conversational', 'Floral', 'Geometric',
       'Graphic', 'Horizontal Stripes', 'Humour and Comic', 'Music', 'People and Places', 'Polka Dots',
       'Self Design', 'Solid', 'Sports', 'Sports and Team Jersey', 'Striped', 'Superhero', 'Tie and Dye',
       'Tribal', 'Typography', 'Varsity'], dtype=object)

In [ ]:
# import random
# import urllib.request

# def download_web_image(url,name):
#     urllib.request.urlretrieve(url,name)

In [ ]:
# temp= df_raw['Sub_category']

In [ ]:
# unique_check = temp.head(n=70000).unique()

In [ ]:
# len(unique_check)

In [ ]:
# unique_check

In [ ]:
# %cd /home/paperspace/data/hackerearth-myntra/train

In [ ]:
%mkdir 'Self Design'

In [ ]:
unique_check

In [ ]:
%cd valid

In [ ]:
%mkdir 'Self Design'

In [ ]:
%ls | wc -l

In [ ]:
%cd /home/paperspace/data/hackerearth-myntra/train

In [ ]:
%cd /home/paperspace/data/hackerearth-myntra/valid

In [ ]:
len(df_raw)

In [ ]:
i = 0
for x in range(0,50000):
    pic_link = df_raw.iloc(0)[i]['Link_to_the_image']
    cat_name = df_raw.iloc(0)[i]['Sub_category']
    print(x)
    i = i+1
    try:
        pic_name_temp = pic_link.split('/')
        pic_name = pic_name_temp[len(pic_name_temp)-1]
        full_path_name = cat_name+'/'+pic_name
        download_web_image(pic_link,full_path_name)
    except:
        print("Some exception")

In [ ]:
?ConvLearner.pretrained

In [ ]:
arch=resnet18
PATH= '/home/paperspace/fastai/courses/dl1/data/hackerearth-myntra/'
bs=8
sz=512
tfms = tfms_from_model(arch, sz)
data = ImageClassifierData.from_paths(PATH,  trn_name='train',val_name='valid',tfms=tfms, bs=bs)
learn = ConvLearner.pretrained(arch, data, precompute=True,ps=0.5)

In [ ]:
lrf = learn.lr_find()

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.sched.plot()

In [ ]:
learn.fit(0.01,1)

In [ ]:
learn.save('0.01_1_512size_resnet18')

In [ ]:
learn.load('0.01_1_512size_resnet18')

In [ ]:
learn.fit(0.001,3)

In [ ]:
learn.unfreeze()

In [ ]:
lr=np.array([1e-4,1e-3,1e-2])

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save('diff_lr_last')

In [ ]:
# learn.load("0.001_3_dnet121_512")

In [ ]:
len(data.trn_y)

In [ ]:
# learn.fit(0.01,3)

In [ ]:
# learn.save("0.01_6_dnet121_256")

In [ ]:
log_preds = learn.predict()
log_preds.shape

In [ ]:
log_preds[:10]

In [ ]:
preds = np.argmax(log_preds, axis=1)  # from log probabilities to 0 or 1
probs = np.exp(log_preds[:,1])  

In [ ]:
def rand_by_mask(mask): return np.random.choice(np.where(mask)[0], 4, replace=False)
def rand_by_correct(is_correct): return rand_by_mask((preds == data.val_y)==is_correct)

In [ ]:
def plot_val_with_title(idxs, title):
    imgs = np.stack([data.val_ds[x][0] for x in idxs])
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(data.val_ds.denorm(imgs), rows=1, titles=title_probs)

In [ ]:
def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

In [ ]:
def load_img_id(ds, idx): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8))

In [ ]:
plot_val_with_title(rand_by_correct(True), "Correctly classified")

In [ ]:
plot_val_with_title(rand_by_correct(False), "Incorrectly classified")